In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katiegarrett/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
top_2017 = pd.read_csv('../data/top_2017.csv')
top_2017.head()

,Unnamed: 0,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,brand_safe,contest_mode,created_utc,domain,...,thumbnail_height,thumbnail_width,media,secure_media,subreddit_type,view_count,link_flair_css_class,link_flair_text,author_cakeday,date
0,0,Jeromiewhalen,1.302417e+09,NaN,NaN,t2_52zmg,True,False,1493865347,self.Teachers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-04 02:35:47+00:00
1,1,gnikzilgnikzil,1.402467e+09,NaN,NaN,t2_gxoif,True,False,1485216564,senate.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-24 00:09:24+00:00
2,2,mustinethegreat,NaN,NaN,NaN,NaN,True,False,1506614977,self.Teachers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-28 16:09:37+00:00
3,3,euqinu99999,1.490578e+09,NaN,NaN,t2_16iuo7,True,False,1496444574,self.Teachers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-02 23:02:54+00:00
4,4,gasoline-rainbows,NaN,NaN,NaN,NaN,True,False,1510005040,yahoo.com,...,93.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06 21:50:40+00:00


In [3]:
top_2017["combined"] = top_2017["title"].astype(str) + top_2017["selftext"].astype(str)

In [4]:
top_2017["row_id"] = top_2017.index + 1 
print (top_2017.head(10))

   Unnamed: 0             author  author_created_utc  author_flair_css_class  \
0           0      Jeromiewhalen        1.302417e+09                     NaN   
1           1     gnikzilgnikzil        1.402467e+09                     NaN   
2           2    mustinethegreat                 NaN                     NaN   
3           3        euqinu99999        1.490578e+09                     NaN   
4           4  gasoline-rainbows                 NaN                     NaN   
5           5     nonsenseword37        1.452627e+09                     NaN   
6           6     happy_bluebird                 NaN                     NaN   
7           7       polkadotgirl                 NaN                     NaN   
8           8         supertizer        1.285323e+09                     NaN   
9           9       michiruwater                 NaN                     NaN   

  author_flair_text author_fullname  brand_safe  contest_mode  created_utc  \
0               NaN        t2_52zmg      

In [5]:
top_2017_subset = top_2017[['row_id', 'combined']].copy()
#data clean-up
#remove all non-aphabet characters
top_2017_subset['combined'] = top_2017_subset['combined'].str.replace("[^a-zA-Z#]", " ")
#convert to lower-case
top_2017_subset['combined'] = top_2017_subset['combined'].str.casefold()
print (top_2017_subset.head(10))

   row_id                                           combined
0       1  jostens yearbooks scammed our high school for ...
1       2  confirmation of betsy devos was delayed  conta...
2       3  not sure if my teacher is drunk or needs medic...
3       4  i got this note from a senior after i decided ...
4       5  officials expect devos to resign from trump ad...
5       6        a sign at the women s march in clevelandnan
6       7  this just passed the senate   members of congr...
7       8  classroom management is everythingedit  whoeve...
8       9   i m still a teacher  and i teach high school nan
9      10  we need to stop subsidizing poor treatment of ...


/var/folders/4_/948td82x7jlf4pz0c4dtqn_00000gn/T/ipykernel_2436/4057735993.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  top_2017_subset['combined'] = top_2017_subset['combined'].str.replace("[^a-zA-Z#]", " ")


In [6]:
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [7]:
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in top_2017_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['row_id']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.row_id != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

Processing sentiment analysis...
   row_id sentiment_type  sentiment_score
0       1       compound           0.9947
0       2       compound          -0.2263
0       3       compound          -0.3753
0       4       compound           0.8807
0       5       compound          -0.3400
0       6       compound           0.0000
0       7       compound           0.0000
0       8       compound           0.9997
0       9       compound           0.0000
0      10       compound          -0.6478


In [8]:
df_output = pd.merge(top_2017, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

   Unnamed: 0             author  author_created_utc  author_flair_css_class  \
0           0      Jeromiewhalen        1.302417e+09                     NaN   
1           1     gnikzilgnikzil        1.402467e+09                     NaN   
2           2    mustinethegreat                 NaN                     NaN   
3           3        euqinu99999        1.490578e+09                     NaN   
4           4  gasoline-rainbows                 NaN                     NaN   
5           5     nonsenseword37        1.452627e+09                     NaN   
6           6     happy_bluebird                 NaN                     NaN   
7           7       polkadotgirl                 NaN                     NaN   
8           8         supertizer        1.285323e+09                     NaN   
9           9       michiruwater                 NaN                     NaN   

  author_flair_text author_fullname  brand_safe  contest_mode  created_utc  \
0               NaN        t2_52zmg      

In [9]:
df_output[["sentiment_score"]].describe()

,sentiment_score
count,726.000000
mean,0.255972
std,0.614631
min,-0.996500
25%,-0.155900
50%,0.301550
75%,0.881375
max,0.999700


In [10]:
top_2017_final = df_output

In [11]:
top_2017_final['year'] = 2017

In [12]:
top_2017_final.head()

,Unnamed: 0,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,brand_safe,contest_mode,created_utc,domain,...,view_count,link_flair_css_class,link_flair_text,author_cakeday,date,combined,row_id,sentiment_type,sentiment_score,year
0,0,Jeromiewhalen,1.302417e+09,NaN,NaN,t2_52zmg,True,False,1493865347,self.Teachers,...,NaN,NaN,NaN,NaN,2017-05-04 02:35:47+00:00,Jostens Yearbooks scammed Our High School for ...,1,compound,0.9947,2017
1,1,gnikzilgnikzil,1.402467e+09,NaN,NaN,t2_gxoif,True,False,1485216564,senate.gov,...,NaN,NaN,NaN,NaN,2017-01-24 00:09:24+00:00,Confirmation of Betsy DeVos was DELAYED! Conta...,2,compound,-0.2263,2017
2,2,mustinethegreat,NaN,NaN,NaN,NaN,True,False,1506614977,self.Teachers,...,NaN,NaN,NaN,NaN,2017-09-28 16:09:37+00:00,Not sure if my teacher is drunk or needs medic...,3,compound,-0.3753,2017
3,3,euqinu99999,1.490578e+09,NaN,NaN,t2_16iuo7,True,False,1496444574,self.Teachers,...,NaN,NaN,NaN,NaN,2017-06-02 23:02:54+00:00,I got this note from a senior after I decided ...,4,compound,0.8807,2017
4,4,gasoline-rainbows,NaN,NaN,NaN,NaN,True,False,1510005040,yahoo.com,...,NaN,NaN,NaN,NaN,2017-11-06 21:50:40+00:00,Officials expect DeVos to resign from Trump ad...,5,compound,-0.3400,2017


In [13]:
top_2017_final = top_2017_final[['author', 'created_utc', 'date', 'combined', 'row_id', 'sentiment_score', 'year']]

In [14]:
top_2017_final.head()

,author,created_utc,date,combined,row_id,sentiment_score,year
0,Jeromiewhalen,1493865347,2017-05-04 02:35:47+00:00,Jostens Yearbooks scammed Our High School for ...,1,0.9947,2017
1,gnikzilgnikzil,1485216564,2017-01-24 00:09:24+00:00,Confirmation of Betsy DeVos was DELAYED! Conta...,2,-0.2263,2017
2,mustinethegreat,1506614977,2017-09-28 16:09:37+00:00,Not sure if my teacher is drunk or needs medic...,3,-0.3753,2017
3,euqinu99999,1496444574,2017-06-02 23:02:54+00:00,I got this note from a senior after I decided ...,4,0.8807,2017
4,gasoline-rainbows,1510005040,2017-11-06 21:50:40+00:00,Officials expect DeVos to resign from Trump ad...,5,-0.3400,2017


In [15]:
top_2017_final.to_csv('top_2017_final.csv')